Import the data from Edge Impulse. You can obtain the URL from the Dashboard, right-click on the download icon next to 'Spectral features data' and 'Spectral features labels', and click **Copy link location**.

In [1]:
import numpy as np
import requests

API_KEY = 'ei_9eedce842a674656748bf65a19f0e2a80cc867cde21a7810354f75a4fb565a3d'

X = (requests.get('https://studio.edgeimpulse.com/v1/api/24007/training/9/x', headers={'x-api-key': API_KEY})).content
Y = (requests.get('https://studio.edgeimpulse.com/v1/api/24007/training/9/y', headers={'x-api-key': API_KEY})).content
with open('x_train.npy', 'wb') as file:
    file.write(X)
with open('y_train.npy', 'wb') as file:
    file.write(Y)

Store the data in a temporary file, and load it back through Numpy.

In [1]:
X = np.load('x_train.npy')
Y = np.load('y_train.npy')[:,0]

Define our labels and split the data up in a test and training set:

In [2]:
import sys, os, random
import tensorflow as tf
from sklearn.model_selection import train_test_split

import logging
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Set random seeds for repeatable results
RANDOM_SEED = 3
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

classes_values = [ "noise", "unknown", "yes" ]
classes = len(classes_values)

Y = tf.keras.utils.to_categorical(Y - 1, classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

input_length = X_train[0].shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

def set_batch_size(batch_size, train_dataset, validation_dataset):
    train_dataset = train_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    validation_dataset = validation_dataset.batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    return train_dataset, validation_dataset

callbacks = []


Train the model:

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Conv1D, Conv2D, Flatten, Reshape, MaxPooling1D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [4]:
# model architecture
model = Sequential()
model.add(Reshape((int(input_length / 13), 13), input_shape=(input_length, )))
model.add(Conv1D(8, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(classes, activation='softmax', name='y_pred'))

# this controls the learning rate
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999)
# this controls the batch size, or you can manipulate the tf.data.Dataset objects yourself
BATCH_SIZE = 32
train_dataset, validation_dataset = set_batch_size(BATCH_SIZE, train_dataset, validation_dataset)

# train the neural network
retrain = False  # if you don't have the weights already
if retrain:
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.fit(train_dataset, epochs=100, validation_data=validation_dataset, verbose=2, callbacks=callbacks)
    
    # Save the model to disk
    model.save('saved_model')
    weights_list = model.get_weights()
    np.savez('weights_float32.npz', *weights_list)
else:  # otherwise reload weights from disk
    weights_archive = np.load('weights_float32.npz')
    weights_list = [weights_archive['arr_{}'.format(i)] for i in range(len(weights_archive))]
    model.set_weights(weights_list)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 50, 13)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 50, 8)             320       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 25, 8)             0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 8)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 25, 16)            400       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 16)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 16)            0

In [5]:
#weights_list = model.get_weights()
for weights in weights_list:
    print(weights.shape)
conv1_weights = weights_list[0]
conv1_biases = weights_list[1]
conv2_weights = weights_list[2]
conv2_biases = weights_list[3]
dense_weights = weights_list[4]
dense_biases = weights_list[5]

(3, 13, 8)
(8,)
(3, 8, 16)
(16,)
(208, 3)
(3,)


In [6]:
print(X.shape)

(3613, 650)


In [29]:
from scipy.special import softmax

def relu(x):
    return np.maximum(x, 0)

def conv1d_single_kernel(x, weights, bias):
    '''Perform convolution of a input feature map with a single filter and bias.
    
    featuremap dims are (time, n_coeffs), so (50, 13)
    '''
    out = np.zeros(x.shape[0], dtype=np.float32)  # output length is the same
    x_pad = np.zeros((x.shape[0]+2, x.shape[1]), dtype=np.float32)
    x_pad[1:-1,:] = x  # zero padding
    for i in range(x.shape[0]):
        section = x_pad[i:i+3,:]
        out[i] = relu(np.sum(section * weights) + bias)
    return out

def conv1d_multi_kernel(x, weights, biases):
    n_kernels = weights.shape[2]
    out = np.zeros((x.shape[0], n_kernels), dtype=np.float32)
    for i in range(n_kernels):
        kernel = weights[:,:,i]
        out[:,i] = conv1d_single_kernel(x, kernel, biases[i])
    return out

def max_pool_1d(x):
    out = np.zeros((int(np.ceil(x.shape[0] / 2)), x.shape[1]), dtype=np.float32)
    for j in range(x.shape[1]):
        for i in range(int(np.floor(x.shape[0] / 2))):
            out[i, j] = np.maximum(x[2*i, j], x[2*i+1, j])
        if (x.shape[0] % 2 == 1):
            out[-1, j] = x[-1, j]
    return out

def dense(x, weights, biases):
    x = x.flatten()
    out = softmax(np.matmul(x, weights) + biases)
    return out

def classify(features):
#     out = np.zeros((features.shape[0], 3), dtype=np.float32)
    out = np.zeros((features.shape[0], 50, 8))
    for i in range(features.shape[0]):
        x = features[i]
        x = x.reshape((int(x.size / 13), 13))
        conv1 = conv1d_multi_kernel(x, conv1_weights, conv1_biases)
#         max_pool1 = max_pool_1d(conv1)
#         conv2 = conv1d_multi_kernel(max_pool1, conv2_weights, conv2_biases)
#         max_pool2 = max_pool_1d(conv2)
#         dense1 = dense(max_pool2, dense_weights, dense_biases)
#         out[i] = dense1
        out[i] = conv1
    return out

In [28]:
my_pred = classify(X[:20,:])
my_pred.shape

ValueError: could not broadcast input array from shape (50,8) into shape (3)

In [9]:
# second copy of model to experiment with
model2 = Sequential()
model2.add(Reshape((int(input_length / 13), 13), input_shape=(input_length, )))
model2.add(Conv1D(8, kernel_size=3, activation='relu', padding='same'))
#model2.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
#model2.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
#model2.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
#model2.add(Flatten())
#model2.add(Dense(classes, activation='softmax', name='y_pred'))
model2.set_weights(weights_list[:2])
#model2.summary()
keras_pred = model2.predict(X[:20,:])

In [10]:
diff = my_pred - keras_pred
diff  # difference should all be small values

ValueError: operands could not be broadcast together with shapes (20,3) (20,50,8) 

In [169]:
import matplotlib.pyplot as plt
plt.imshow(diff)

In [12]:
# print(conv1_weights.shape)
# scale = 0.014854089356958866
# zero_point = 0
# quant = conv1_weights[:,:,0] / scale + zero_point
# quant.flatten().astype(int)

In [11]:
# print(dense_weights.shape)
# scale = 0.0089818863198161125
# quant = dense_weights / scale
# quant.astype(int)

In [13]:
#model.evaluate(X, Y)

In [15]:
def representative_data_gen():
    n_samples = 100
    for input_value in tf.data.Dataset.from_tensor_slices(X).batch(1).take(100):
        # Model has only one input so each data point has one element.
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model2)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model_quant = converter.convert()

In [16]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [39]:
tensor_details = interpreter.get_tensor_details()
tensor_details

[{'name': 'reshape_1_input_int8',
  'index': 0,
  'shape': array([  1, 650], dtype=int32),
  'shape_signature': array([ -1, 650], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.036910686641931534, -9),
  'quantization_parameters': {'scales': array([0.03691069], dtype=float32),
   'zero_points': array([-9], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'sequential_1/conv1d_2/BiasAdd/ReadVariableOp/resource',
  'index': 1,
  'shape': array([8], dtype=int32),
  'shape_signature': array([8], dtype=int32),
  'dtype': numpy.int32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([0.00051168, 0.0002173 , 0.00025702, 0.00027215, 0.00028592,
          0.00025434, 0.00027363, 0.00034526], dtype=float32),
   'zero_points': array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'sequential_1/conv1d_2/conv1d/ExpandDims/dim',
  'index': 2,
  'shape': array([], dtype=

In [22]:
import pathlib

tflite_model_quant_file = pathlib.Path('int8-model.tflite')
tflite_model_quant_file.write_bytes(tflite_model_quant)

In [18]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = X[test_image_index]
    test_label = Y[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    
    return interpreter.get_tensor(output_details["index"])[0]  # TODO: remove

    predictions[i] = output.argmax()

  return predictions

In [19]:
def evaluate_model(tflite_file, model_type):
  test_mfcc_indices = range(X.shape[0])
  predictions = run_tflite_model(tflite_file, test_mfcc_indices)

  accuracy = (np.sum(np.argmax(Y, axis=1) == predictions) * 100) / len(X)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(X)))

In [74]:
evaluate_model(tflite_model_quant_file, model_type="Quantized")

Quantized model accuracy is 96.4296% (Number of test samples=3613)


[Reference Implementation](https://github.com/tensorflow/tensorflow/blob/92f84571edb98bdb31f1e7b1e23f17bf2c1438e9/tensorflow/lite/kernels/internal/reference/conv.h#L101)

In [24]:
input_details = interpreter.get_input_details()[0]
input_scale, input_zero_point = input_details["quantization"]

conv1_weight_details = tensor_details[5]['quantization_parameters']
conv1_weight_scale = conv1_weight_details['scales']
conv1_weight_zero = conv1_weight_details['zero_points']

conv1_biases_details = tensor_details[1]['quantization_parameters']
conv1_biases_scale = conv1_biases_details['scales']
conv1_biases_zero = conv1_biases_details['zero_points']

conv1_activation_details = tensor_details[14]['quantization_parameters']
conv1_activation_scale = conv1_activation_details['scales']
conv1_activation_zero = conv1_activation_details['zero_points']
conv1_activation_max = 8.34693717956543
conv1_activation_min = 0

quant_conv1_weights = np.zeros(conv1_weights.shape)
quant_conv1_biases = np.zeros(conv1_biases.shape)
for i in range(conv1_biases.size):
    quant_conv1_weights[:,:,i] = conv1_weights[:,:,i] / conv1_weight_scale[i] + conv1_weight_zero[i]
    quant_conv1_biases[i] = conv1_biases[i] / conv1_biases_scale[i] + conv1_biases_zero[i]

def quant_relu(x):
    return np.maximum(x, 0)

def quant_conv1d_single_kernel(x, weights, bias, x_scale, x_zero, kernel_scale, kernel_zero,
                               activation_scale, activation_zero):
    '''Perform convolution of a input feature map with a single filter and bias.
    
    featuremap dims are (time, n_coeffs), so (50, 13)
    '''
    out = np.zeros(x.shape[0], dtype=np.float32)  # output length is the same
    x_pad = np.zeros((x.shape[0]+2, x.shape[1]), dtype=np.float32)
    x_pad[1:-1,:] = x  # zero padding
    for i in range(x.shape[0]):
        section = x_pad[i:i+3,:]
        #out[i] = quant_relu(np.sum(section * weights) + bias)
        #acc = np.sum((section - x_zero) * (weights - kernel_zero))
        acc = np.sum((section - x_zero) * weights)
        acc += bias
        multiplier = x_scale * kernel_scale / activation_scale  # convert to quantized multiplier
        preactivation = multiplier * acc + activation_zero
        #out[i] np.clip(activated, activation_min, activation_max)
        #import ipdb
        #ipdb.set_trace()
        #out[i] = quant_relu(preactivation)
        out[i] = preactivation
    return out

def quant_conv1d_multi_kernel(x, weights, biases, x_scale, x_zero, weight_scale, weight_zero,
                              activation_scale, activation_zero):
    n_kernels = weights.shape[2]
    out = np.zeros((x.shape[0], n_kernels), dtype=np.float32)
    for i in range(n_kernels):
        kernel = weights[:,:,i]
        kernel_scale = weight_scale[i]
        kernel_zero = weight_zero[i]
        out[:,i] = quant_conv1d_single_kernel(x, kernel, biases[i], x_scale, x_zero, kernel_scale, kernel_zero,
                                              activation_scale, activation_zero)
    return out

def quant_max_pool_1d(x):
    out = np.zeros((int(np.ceil(x.shape[0] / 2)), x.shape[1]), dtype=np.float32)
    for j in range(x.shape[1]):
        for i in range(int(np.floor(x.shape[0] / 2))):
            out[i, j] = np.maximum(x[2*i, j], x[2*i+1, j])
        if (x.shape[0] % 2 == 1):
            out[-1, j] = x[-1, j]
    return out

def quant_dense(x, weights, biases):
    x = x.flatten()
    out = softmax(np.matmul(x, weights) + biases)
    return out

def quant_classify(features):
#     out = np.zeros((features.shape[0], 3), dtype=np.float32)
    out = np.zeros((features.shape[0], 50, 13))
    for i in range(features.shape[0]):
        x = features[i]
        x = x / input_scale + input_zero_point
        x = x.reshape((int(x.size / 13), 13))
        conv1 = quant_conv1d_multi_kernel(x, quant_conv1_weights, quant_conv1_biases,
                                          input_scale, input_zero_point,
                                          conv1_weight_scale, conv1_weight_zero,
                                          conv1_activation_scale, conv1_activation_zero)
#         max_pool1 = quant_max_pool_1d(conv1)
#         conv2 = quant_conv1d_multi_kernel(max_pool1, conv2_weights, conv2_biases)
#         max_pool2 = quant_max_pool_1d(conv2)
#         dense1 = quant_dense(max_pool2, dense_weights, dense_biases)
#         out[i] = dense1
        out = conv1
    return out

In [38]:
tflite_result = run_tflite_model(tflite_model_quant_file, [0])
numpy_result = quant_classify(X[:1])
my_pred = classify(X[:1,:])
numpy_result.astype(int)
#tflite_result

array([[-149, -289, -140, -328, -385, -491,    7,  -39],
       [-191, -188, -293, -332, -266, -239, -118, -122],
       [-175, -173, -304, -271, -230, -209, -129,  -99],
       [-128, -232, -277, -272, -231, -235, -160,  -62],
       [-163, -229, -268, -296, -285, -205, -150, -107],
       [-159, -230, -300, -273, -238, -189, -154, -111],
       [-153, -189, -232, -248, -253, -264, -193,  -88],
       [-165,  -91, -170, -215, -232, -250,  -68, -125],
       [-122,  -82, -159, -228, -180, -220, -126, -134],
       [-131,  -60, -191, -252, -168, -177, -171, -187],
       [-172,  -78, -157, -265, -187, -201, -123, -149],
       [-173,  -84, -186, -255, -193, -191, -154, -170],
       [-157, -133, -279, -256, -183, -173, -150, -124],
       [-136, -121, -245, -168, -200, -162, -208, -154],
       [-109, -135, -116, -115,  -92, -132, -188, -172],
       [-160, -250,  -18, -193,  -76, -121, -198, -102],
       [-144, -230,   11, -201, -146,  -72, -238, -152],
       [-132, -270,  -28, -181,

In [37]:
tflite_result
np.logical_and((my_pred==0), (tflite_result==0))

array([[[ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True, False],
        [ True, False,  True,  True,  True,  True, False, False],
        [False, False,  True,  True,  True,  True, False,  True],
        [ True, False,  True,  True,  True,  True,  True,  True],
        [ True, False,  True,  True,  True,  True, False,  True],
        [ True, False,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True, False],
        [ True, False,  True,  True,  True,  True,  True,  True],
        [False,  True, False, False, False,  True,  True,  True],
        [ 

In [26]:
conv1_activation_zero

array([-128], dtype=int32)

In [141]:
quant_conv1_weights

array([[[-127.        ,   59.84857559,   80.46018219, -110.07582092,
          127.        ,  126.99999237,  -54.62316895, -126.99999237],
        [  21.9630146 ,   98.51624298,  -32.76836014,   24.05471992,
          -14.2999773 ,   -8.27717018,  -29.89610481,  -37.39074326],
        [  30.40066719,   39.2250824 ,  -36.42618179,   -8.2433691 ,
           -2.4023962 ,  -19.72091484,  -69.71202087,    2.03873706],
        [  -7.3085947 ,  -38.56336594,  -28.49256134,    9.91439533,
          -31.11902809,   20.89669418,  -56.71468735,   42.77544403],
        [  14.22246933,   15.38040924,  -87.23403168,   96.75441742,
           17.87460518,   35.59344482,   43.77444077,  -27.10764313],
        [  -1.02952313,   14.82881737,   11.56383419,   -9.6651001 ,
           23.94236755,   20.30063248,   -7.64687967,    7.2720747 ],
        [  19.92416763,  -23.60510254,  -14.15288162,  -43.06171799,
           15.97456551,   61.60734177,    8.99171638,   25.44936943],
        [   8.38902664,  -2

In [131]:
conv1_weight_details

{'scales': array([0.01386269, 0.00588729, 0.00696319, 0.00737325, 0.00774639,
        0.00689075, 0.00741341, 0.00935394], dtype=float32),
 'zero_points': array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 'quantized_dimension': 0}

In [133]:
conv1_biases.shape